In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
from math import floor
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from datetime import timedelta

In [2]:
df = pd.read_csv('chart.csv')

In [3]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [4]:
df = df.sort_values(by='Date', ascending=True)
df.reset_index(inplace=True, drop=True)

In [5]:
starts = []
ends = []
deltas = []
start_date = []

for k in range(df.shape[0]):
    start = df['Adj Close'][k]
    end = df['Adj Close'][df.shape[0]-1]
    delta = (end-start)/start

    starts.append(start)
    ends.append(end)
    deltas.append(delta)
    start_date.append(df['Date'][k])

delta_df = pd.DataFrame({'Date':start_date, 'start':starts, 'end':ends, 'delta':deltas})

In [6]:
lump_sum = 10000
values = []
n_etfs = []

for k in range(df.shape[0]):
    start = df['Adj Close'][k]
    n_etf = floor(lump_sum/start)
    temp_values = []
    for c in range(df.shape[0]):
        if df['Date'][c] < df['Date'][k]:
            value = np.nan
        else:
            value = n_etf * df['Adj Close'][c]
        temp_values.append(value)
    values.append([temp_values])
    n_etfs.append(n_etf)

In [7]:
from random import sample
i_2012 = df[df['Date'].dt.year == 2012].index.values
i_2013 = df[df['Date'].dt.year == 2013].index.values
i_2014 = df[df['Date'].dt.year == 2014].index.values
i_2015 = df[df['Date'].dt.year == 2015].index.values
i_2016 = df[df['Date'].dt.year == 2016].index.values
i_2017 = df[df['Date'].dt.year == 2017].index.values
i_2018 = df[df['Date'].dt.year == 2018].index.values
i_2019 = df[df['Date'].dt.year == 2019].index.values
i_2020 = df[df['Date'].dt.year == 2020].index.values
i_2021 = df[df['Date'].dt.year == 2021].index.values
i_2022 = df[df['Date'].dt.year == 2022].index.values
i_2023 = df[df['Date'].dt.year == 2023].index.values

indexes = {'2012':i_2012, '2013':i_2013, '2014':i_2014, '2015':i_2015, '2016':i_2016, '2017':i_2017, 
           '2018':i_2018, '2019':i_2019, '2020':i_2020, '2021':i_2021, '2022':i_2022, '2023':i_2023,}

In [8]:
app = dash.Dash(__name__)

years = list(indexes.keys())

app.layout = html.Div([
    dcc.Dropdown(
        id='value-dropdown',
        options=[{'label': str(value), 'value': value} for value in years],
        value=years[0]
    ),
    html.Button('Randomize', id='randomize-button', n_clicks=0),
    dcc.Graph(id='line-plot')
])

@app.callback(
    Output('line-plot', 'figure'),
    Input('randomize-button', 'n_clicks'),
    Input('value-dropdown', 'value')
)
def update_line_plot(n_clicks, selected_value):
    random_a = np.random.choice(indexes[selected_value], size=1)[0]
    random_b = np.random.choice(indexes[selected_value], size=1)[0]
    random_c = np.random.choice(indexes[selected_value], size=1)[0]
    random_d = np.random.choice(indexes[selected_value], size=1)[0]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'], y=values[random_a][0], mode='lines', name=str(df['Date'][random_a].date())))
    fig.add_trace(go.Scatter(x=df['Date'], y=values[random_b][0], mode='lines', name=str(df['Date'][random_b].date())))
    fig.add_trace(go.Scatter(x=df['Date'], y=values[random_c][0], mode='lines', name=str(df['Date'][random_c].date())))
    fig.add_trace(go.Scatter(x=df['Date'], y=values[random_d][0], mode='lines', name=str(df['Date'][random_d].date())))

    fig.update_layout(title=f'Sample of Lump Sum Investments of 10k € in MSCI World ETF during year {selected_value}')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [9]:
import plotly.express as px

fig = px.line(delta_df, x='Date', y='delta')

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Delta",
    title="Delta Returns of Investments in MSCI World ETF",
)

# Adding the red dotted line at y = 0
fig.add_shape(
    type="line",
    x0=delta_df['Date'].min(),
    x1=delta_df['Date'].max(),
    y0=0,
    y1=0,
    line=dict(color="red", width=2)
)

fig.show()

In [10]:
df = df[df['Date'].dt.year >= 2012]
df.reset_index(drop = True, inplace = True)

In [11]:
dates = []
prices = []
start = 514
for k in range(start, df.shape[0], 30):
    c = 1
    date = df['Date'][0] + timedelta(days = k)
    if k == start:
        price = df[df['Date'] == date]['Close'].values[0]
        prices.append(price)
        dates.append(date)
        counter = 0
    elif k == start + 30:
        while date not in df['Date'].values or date.month == dates[counter-1]:
            date = date + timedelta(days = c)
            c = c+1
        price = df[df['Date'] == date]['Close'].values[0]
        prices.append(price)
        dates.append(date)
    elif k >= start + 60:
        while date not in df['Date'].values or date.month == dates[counter-1] or date.month == dates[counter-2]:
            date = date + timedelta(days = c)
            c = c+1
        price = df[df['Date'] == date]['Close'].values[0]
        prices.append(price)
        dates.append(date)
    counter = counter + 1

In [12]:
dca_values = []
qties = []
deposit = floor(10000/len(dates))
#convert to len and if 0...
for k in range(len(prices)):
    qty = floor(deposit/prices[k])
    if k == 0:
        qties.append(qty)
    else:
        qties.append(qty + qties[k-1])
    dca_values.append(qties[k]*prices[k])

In [14]:
lumpsum_values = []
price = df[df['Date'] == dates[0]]['Close'].values[0]
lump_qty = floor(10000/price)
for k in prices:
    lumpsum_values.append(lump_qty*k)

In [52]:
def update_plot(df, selected_date):
    position = np.where(df['Date'] == selected_date)[0][0]
    dates = []
    prices = []
    last = df['Date'][df.shape[0]-1] - timedelta(days = 30)
    for k in range(position, df.shape[0], 30):
        c = 1
        if k == position:
            date = df['Date'][k]
            start = df['Date'][k]
            price = df[df['Date'] == date]['Close'].values[0]
            prices.append(price)
            dates.append(date)
            counter = 0
        elif k == position + 30:
            date = start + timedelta(days = k)
            while date not in df['Date'].values or date.month == dates[counter-1]:
                date = date + timedelta(days = c)
                c = c+1
            price = df[df['Date'] == date]['Close'].values[0]
            prices.append(price)
            dates.append(date)
        elif k >= position + 60:
            date = start + timedelta(days = k)
            while date not in df['Date'].values or date.month == dates[counter-1] or date.month == dates[counter-2]:
                date = date + timedelta(days = c)
                c = c+1
                if date > last:
                    break
            if date in df['Date']:
                price = df[df['Date'] == date]['Close'].values[0]
                prices.append(price)
                dates.append(date)
                counter = counter + 1
        print(dates)

    dca_values = []
    qties = []
    deposit = floor(10000/len(dates))

    for k in range(len(prices)):
        qty = floor(deposit/prices[k])
        if k == 0:
            qties.append(qty)
        else:
            qties.append(qty + qties[k-1])
        dca_values.append(qties[k]*prices[k])

    lumpsum_values = []
    price = df[df['Date'] == dates[0]]['Close'].values[0]
    lump_qty = floor(10000/price)
    for k in prices:
        lumpsum_values.append(lump_qty*k)
    
    results = {'dates': dates, 'dca_values': dca_values, 'lumpsum_values': lumpsum_values}
    
    return(results)


In [16]:
import plotly.express as px

# Assuming df is your DataFrame with 'Date' and 'Close' columns
line_plot = px.line(df, x='Date', y='Close', title='Close Prices Over Time')

# Update tooltip format
line_plot.update_traces(
    hovertemplate='Date: %{x|%Y-%m-%d}<br>Close: %{y:.3f}'
)

line_plot.show()

In [130]:
def preprocess(df, start_date):
    position = np.where(df['Date'] == start_date)[0][0]
    temp_df = df[df.index >= position].reset_index(drop = True)
    prova = np.unique(temp_df['Date'].dt.to_period('M'))
    dates = []
    for k in prova:
        c = 0
        date = k.to_timestamp() + pd.DateOffset(day - 1)
        while date not in temp_df['Date'].values and date < temp_df['Date'][temp_df.shape[0]-1]:
            if date - timedelta(days = c) in temp_df['Date'].values:
                date = date - timedelta(days = c)
            else:
                date = date + timedelta(days = c)
            c = c+1
        dates.append(date)

    temp_df = temp_df[temp_df['Date'].isin(dates)]
    
    return(temp_df)

def invest(temp_df, tot_money, start_date):

    temp_df = update_plot(df, start_date = start_date)

    quota = floor(tot_money/temp_df.shape[0])

    dca_qty = []
    dca_value = []

    lump_qty = floor(tot_money/temp_df['Adj Close'].values[0])
    lump_value = []

    for k in range(len(temp_df['Adj Close'])):
        bought_qty = floor(quota/temp_df['Adj Close'].values[k])
        dca_qty.append(bought_qty)
        dca_value.append(sum(dca_qty) * temp_df['Adj Close'].values[k])
        lump_value.append(lump_qty * temp_df['Adj Close'].values[k])

    temp_df['dca_value'] = dca_value
    temp_df['lump_value'] = lump_value

    return(temp_df)


def compare_investing(df, start_date, tot_money):
    temp_df = preprocess(df, start_date)
    final_df = invest(temp_df, tot_money, start_date)
    return(final_df)

In [133]:
final_df = compare_investing(df, start_date = '2015-02-06', tot_money = 100000) #choose here the date and the money to invest

trace_dca = go.Scatter(x=final_df['Date'], y=final_df['dca_value'], mode='lines', name='DCA Values')
trace_lumpsum = go.Scatter(x=final_df['Date'], y=final_df['lump_value'], mode='lines', name='Lumpsum Values')

# Create layout for the plot
layout = go.Layout(title='DCA vs Lumpsum Values Over Time',
                   xaxis=dict(title='Dates'),
                   yaxis=dict(title='Values'))

# Create the figure
fig = go.Figure(data=[trace_dca, trace_lumpsum], layout=layout)

# Display the plot
fig.show()